In [3]:
import os
import unittest
import requests
import urllib3
from utils_demo import *
from loguru import logger
from unittest.mock import patch
from pprint import pprint
from os.path import dirname, abspath
import warnings
warnings.filterwarnings('ignore')
sys.path.append('../../') 
from app_decomposer import DEFAULT_CONFIGURATION, KIWI_CONFIG, CURRENT_DIR, API_DICT_TS, IOI_SAMPLING_PERIOD, PERF_MODEL_DATASET_NAME
from app_decomposer.api_connector import request_delegator
from app_decomposer.config_parser import Configuration
from app_decomposer.api_connector import TimeSeries
from performance_model.performance_model import PerformanceModel

ImportError: cannot import name 'PerformanceModel' from 'performance_model.performance_model' (/home_nfs/mimounis/iosea-wp3-recommandation-system/app_decomposer/notebooks/../../performance_model/performance_model/__init__.py)

In [3]:
# Connect directly to kiwi0
#%%capture
logger.remove()
ioi_config = Configuration(path=KIWI_CONFIG)
job_id = 6860 #5168
cd = ComplexDecomposer(job_id=job_id, v0_threshold=1e-8, config=ioi_config)
# Launch decomposition on the signal
representation = cd.get_job_representation(merge_clusters=True)
pprint(representation)
# This is the app encoding representation for Execution Simulator
pprint(f"compute={representation['events']}, reads={representation['read_volumes']}" 
      f"read_bw={representation['read_bw']}, writes={representation['write_volumes']}"
      f"write_bw={representation['write_bw']}, read_pattern={representation['read_pattern']}")
# Normalize signals to seconds and MB
timestamps = (cd.timestamps - cd.timestamps[0])/5
original_read = cd.read_signal/1e6
original_write = cd.write_signal/1e6

fig = plot_detected_phases(jobid=job_id, merge=True, show_phases=True, 
                           ts=(timestamps, original_read, original_write),
                           width=1200, height=600)
fig.show() 


['curl', '--silent', '--insecure', '--header', 'Content-Type: application/x-www-form-urlencoded', '--request', 'POST', '--data', 'username=ioi-admin', '--data', 'password=password', '--data', 'grant_type=password', '--data', 'client_id=io-instrumentation', 'http://kiwi0:8080/auth/realms/atos-data-management/protocol/openid-connect/token']
{'events': [0, 1, 13, 25, 37, 67, 97, 116],
 'node_count': 1,
 'read_bw': [1694.0,
             476625591.1818182,
             26214401227.5,
             20971520383.0,
             782.0,
             506.0,
             0,
             0],
 'read_operations': [5, 10004, 10004, 10004, 2, 2, 0, 0],
 'read_pattern': ['Uncl', 'Seq', 'Seq', 'Seq', 'Uncl', 'Uncl', 'Uncl', 'Uncl'],
 'read_volumes': [1694, 10485763006, 52428802455, 104857601915, 782, 506, 0, 0],
 'write_bw': [0, 0, 0, 0, 5242880000.0, 17476266666.666668, 26214400000.0, 0],
 'write_operations': [0, 0, 0, 0, 10000, 10000, 10000, 0],
 'write_pattern': ['Uncl', 'Uncl', 'Uncl', 'Uncl', 'Seq', 

In [13]:
# Extract phases table and features
phases_features = cd.get_phases_features(representation,
                                         job_id = job_id,
                                         update_csv=True)

# display the csv table
csv_path = os.path.join(dirname(dirname(CURRENT_DIR)), "performance_model", "dataset", PERF_MODEL_DATASET_NAME)
print(pd.read_csv(csv_path, index_col=False))

    job_id        volume   mode IOpattern        IOsize  nodes        ioi_bw
0     5168          6105   read      uncl  3.591176e+02      1  1.221000e+03
1     5168    3145733064   read       seq  1.044400e+06      1  4.839589e+07
2     5168    5242880997   read       seq  5.221993e+06      1  1.048576e+09
3     5168   10485761994   read       seq  5.221993e+06      1  1.048576e+09
4     5168   10485760317   read       seq  1.044398e+07      1  2.097152e+09
5     5168   20971520634   read       seq  1.044398e+07      1  2.097152e+09
6     5168     307200000  write       seq  1.024000e+05      1  6.144000e+07
7     5168    3145728000  write       seq  1.048576e+06      1  3.145728e+08
8     5168   31457280000  write       seq  1.048576e+07      1  4.194304e+08
9     5168             0   read      uncl  0.000000e+00      1  0.000000e+00
10    6860          1694   read      uncl  3.388000e+02      1  3.388000e+02
11    6860   10485763006   read       seq  1.048157e+06      1  9.532512e+07

In [ ]:
# Feed three columns of the CSV table by launching tiers perf measurements
# with nfw_bw | lfs_bw | sbb_bw



In [ ]:
# Apply same BW as measured by IOI (from representation)
read_bw = list(map(lambda x: x/1e6, representation['read_bw']))
write_bw = list(map(lambda x: x/1e6, representation['write_bw']))
ioi_bw = list(map(lambda x, y: (x + y), read_bw, write_bw))
print(ioi_bw)

In [ ]:
# Get the BW predicted if the app run on some tiers // no model, just indexed tabular
PERF_MODEL_DATASET_NAME = "performance_model_dataset_completed_2.csv"
current_dir = dirname(dirname(os.getcwd()))
csv_path = os.path.join(current_dir, "performance_model", "dataset",
                        PERF_MODEL_DATASET_NAME)
df = pd.read_csv(csv_path, index_col=False)
perf = {}
for tier_bw in ["ioi_bw", "nfs_bw", "lfs_bw", "sbb_bw"]:
    rough_bw = list(df[[tier_bw]].to_numpy().flatten())
    # perf adjusted to IOI
    perf[tier_bw] = list(map(lambda x: x*5/1e6, rough_bw))       
  
pprint(perf)


In [ ]:
# get the ground truth = experimental job of the same app running the the specified tier for verification.

# 5168-5175: nfs
# 5182-5184: fs1
# 5192-5194: sbb

timestamps, experimental_read, experimental_write = get_job_timeseries_from_file_as_array(job_id=5168)
timestamps = (timestamps - timestamps[0])/5
experimental_read = experimental_read/1e6
experimental_write = experimental_write/1e6

In [ ]:
sim_time, sim_read_bw, sim_write_bw = simulate_app(representation['events'],
                                                   representation['read_volumes'],
                                                   representation['write_volumes'], 
                                                   perf["ioi_bw"], app_name="job#5192")


fig = display_original_sim_signals((sim_time, sim_read_bw, sim_write_bw),
                                   (timestamps, experimental_read, experimental_write),
                                   str_org="Experimental (same tier)",
                                   str_sim="Predicted",
                                   width=1000, height=900)
fig.show()